# Checking PyPSA and Gurobi installation

This is a very simple script to test that PyPSA and Gurobi installations are working properly.

It is based on the example "Demonstrate PyPSA unit commitment with a one-bus two-generator" available as a Jupyter notebook at http://www.pypsa.org/examples/unit-commitment.ipynb



In [8]:
import pypsa

Initiallize the network, set 4 time steps in the snapshots.

In [9]:
network = pypsa.Network()

network.set_snapshots(range(4))

Add one bus named "bus" to the network and add two generators (coal and gas).

Include the data regarding nominal power (p_nom), marginal cost (marginal_cost), and minimum power (p_min_pu) that needs to be dispatched by every generator. To enable unit commitment on a generator, set its attribute committable = True.

Add the load to the bus.

In [3]:
network.add("Bus","bus")


network.add("Generator",
            "coal",
            bus="bus",
            committable=True,
            p_min_pu=0.3,
            marginal_cost=20,
            p_nom=10000)

network.add("Generator",
            "gas",
            bus="bus",
            committable=True,
            marginal_cost=70,
            p_min_pu=0.1,
            p_nom=1000)

network.add("Load",
            "load",
            bus="bus",
            p_set=[4000,6000,5000,800])

Solve the linear optimal power flow (lopf) using Gurobi as solver.

In this case, we are optimising the dispatch of every generator to minimise the total system cost.

We use the option 'pyomo=False' which means that the optimization problem is directly passed from PyPSA to Gurobi, without using pyomo as an interface. 

In [4]:
network.lopf(network.snapshots, 
             pyomo=False,
             solver_name='gurobi')

INFO:pypsa.linopf:Prepare linear problem
INFO:pypsa.linopf:Total preparation time: 0.36s
INFO:pypsa.linopf:Solve linear problem using Gurobi solver


Academic license - for non-commercial use only - expires 2022-08-22
Using license file C:\Users\34620\gurobi.lic
Read LP format model from file C:\Users\34620\AppData\Local\Temp\pypsa-problem-j8futwr6.lp
Reading time = 0.03 seconds
obj: 20 rows, 17 columns, 40 nonzeros
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 20 rows, 17 columns and 40 nonzeros
Model fingerprint: 0x0e50e8b9
Variable types: 9 continuous, 8 integer (8 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+00, 7e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e+02, 6e+03]
Presolve removed 20 rows and 17 columns
Presolve time: 0.03s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.07 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 356000 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.5600000

INFO:pypsa.linopf:Optimization successful. Objective value: 3.56e+05


('ok', 'optimal')

The result ('Optimal solution found') indicates that the optimizer has found an optimal solution. Hence, you have checked that it is working. 

Now, you can for example show the status of every generator (if they are working or not)

In [5]:
network.generators_t.status

,coal,gas
snapshot,,
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0
3,0.0,1.0


Show the power produced by every generator 

In the final hour, load goes below part-load limit of coal generation (30%), forcing gas to commit.

In [6]:
network.generators_t.p

,coal,gas
snapshot,,
0,4000.0,0.0
1,6000.0,0.0
2,5000.0,0.0
3,0.0,800.0


The total cost can be read from the network objetive.

In [7]:
network.objective

356000.0